This notebook, I will manually copy and paste the output of "# Step by Step debugging the collating function before apply to chat template (to be deleted)" section from fine_tuning_vlm_for_object_detection_trl.ipynb

The structure: I will copy the code over and put the output in the markdown right after the code cell.

In [ ]:
print(train_dataset_formatted[0])
print(type(train_dataset_formatted))
print(type(train_dataset_formatted[0]))
print(type([train_dataset_formatted[0:10]]))

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2592x1944 at 0x73B60D691480>, 'messages': [{'content': [{'image': None, 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.', 'type': 'text'}], 'role': 'system'}, {'content': [{'image': 'IMAGE_PLACEHOLDER', 'text': None, 'type': 'image'}, {'image': None, 'text': 'Detect the bounding box of the nutrition table.', 'type': 'text'}], 'role': 'user'}, {'content': [{'image': None, 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(14,57),(991,603)<|box_end|>', 'type': 'text'}], 'role': 'assistant'}]}
<class 'datasets.arrow_dataset.Dataset'>
<class 'dict'>
<class 'list'>

In [ ]:
batch = train_dataset_formatted

# Extract messages and images from each sample
messages_list = [sample['messages'] for sample in batch]
images_list = [sample.get('image', None) for sample in batch]



In [ ]:
print(type(messages_list))
print(type(images_list))
# print(messages_list)
# print(images_list)
for i in range(5):
    print(messages_list[i])
    print(images_list[i])
    print("-"*100)

<class 'list'>
<class 'list'>
[{'content': [{'image': None, 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.', 'type': 'text'}], 'role': 'system'}, {'content': [{'image': 'IMAGE_PLACEHOLDER', 'text': None, 'type': 'image'}, {'image': None, 'text': 'Detect the bounding box of the nutrition table.', 'type': 'text'}], 'role': 'user'}, {'content': [{'image': None, 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(14,57),(991,603)<|box_end|>', 'type': 'text'}], 'role': 'assistant'}]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2592x1944 at 0x73B69184BA60>
----------------------------------------------------------------------------------------------------
[{'content': [{'image': None, 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.', 'type': 'text'}], 'role': 'system'}, {'content': [{'image': 'IMAGE_PLACEHOLDER', 'text': None, 'type': 'image'}, {'image': None, 'text': 'Detect the bounding box of the nutrition table.', 'type': 'text'}], 'role': 'user'}, {'content': [{'image': None, 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(147,151),(516,588)<|box_end|>', 'type': 'text'}], 'role': 'assistant'}]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=306x408 at 0x73B69184B790>
----------------------------------------------------------------------------------------------------
[{'content': [{'image': None, 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.', 'type': 'text'}], 'role': 'system'}, {'content': [{'image': 'IMAGE_PLACEHOLDER', 'text': None, 'type': 'image'}, {'image': None, 'text': 'Detect the bounding box of the nutrition table.', 'type': 'text'}], 'role': 'user'}, {'content': [{'image': None, 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(10,8),(978,994)<|box_end|>', 'type': 'text'}], 'role': 'assistant'}]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=692x720 at 0x73B69184B4C0>
----------------------------------------------------------------------------------------------------
[{'content': [{'image': None, 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.', 'type': 'text'}], 'role': 'system'}, {'content': [{'image': 'IMAGE_PLACEHOLDER', 'text': None, 'type': 'image'}, {'image': None, 'text': 'Detect the bounding box of the nutrition table.', 'type': 'text'}], 'role': 'user'}, {'content': [{'image': None, 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(320,171),(627,625)<|box_end|>', 'type': 'text'}], 'role': 'assistant'}]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3120x4208 at 0x73B69184B880>
----------------------------------------------------------------------------------------------------
[{'content': [{'image': None, 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.', 'type': 'text'}], 'role': 'system'}, {'content': [{'image': 'IMAGE_PLACEHOLDER', 'text': None, 'type': 'image'}, {'image': None, 'text': 'Detect the bounding box of the nutrition table.', 'type': 'text'}], 'role': 'user'}, {'content': [{'image': None, 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(143,251),(376,468)<|box_end|>', 'type': 'text'}], 'role': 'assistant'}]
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1944x2592 at 0x73B69184B310>
----------------------------------------------------------------------------------------------------

In [ ]:
print(len(messages_list))

1083

In [ ]:
# Filter out samples without images
valid_pairs = [(m, img) for m, img in zip(messages_list, images_list) if img is not None]
if not valid_pairs:
    raise ValueError("Batch contains no valid images.")

messages_list, images_list = zip(*valid_pairs)
messages_list = list(messages_list)
images_list = list(images_list)

# Process each sample
texts = []
all_images = []



In [ ]:
all_conversations = []  # This will hold complete conversations

for messages, image in zip(messages_list, images_list):
    messages_with_image = []
    # Clean up messages: remove None values and restore images
    for msg in messages:
        msg_copy = {'role': msg['role'], 'content': []}
        
        for content_item in msg['content']:
            # Skip None entries entirely
            if content_item is None:
                continue
                
            # Process text content - filter out None text values
            if content_item.get('type') == 'text':
                text_value = content_item.get('text')
                if text_value is not None and text_value != 'None':  # Check for actual None and string 'None'
                    msg_copy['content'].append({
                        'type': 'text',
                        'text': text_value
                    })
            # Process image content - replace IMAGE_PLACEHOLDER with actual PIL image
            elif content_item.get('type') == 'image' and msg['role'] == 'user':
                # Check if it's IMAGE_PLACEHOLDER and replace with actual image
                image_value = content_item.get('image')
                if image_value == 'IMAGE_PLACEHOLDER' or image_value is None:
                    # Replace with the actual PIL image from top level
                    msg_copy['content'].append({
                        'type': 'image',
                        'image': image  # Use the actual PIL image
                    })
                elif image_value and image_value != 'None':
                    # Use existing image if it's not placeholder
                    msg_copy['content'].append({
                        'type': 'image',
                        'image': image_value
                    })
        
        if msg_copy['content']:
            messages_with_image.append(msg_copy)

    # Add the complete conversation (3 messages) to collection
    all_conversations.append(messages_with_image)

print("First 5 messages_with_image examples:")
for i in range(min(5, len(all_conversations))):
    print(f"\nExample {i}:")
    print(all_conversations[i])

First 5 messages_with_image examples:

Example 0:
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2592x1944 at 0x73B69184BA60>}, {'type': 'text', 'text': 'Detect the bounding box of the nutrition table.'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(14,57),(991,603)<|box_end|>'}]}]

Example 1:
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=306x408 at 0x73B69184B790>}, {'type': 'text', 'text': 'Detect the bounding box of the nutrition table.'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(147,151),(516,588)<|box_end|>'}]}]

Example 2:
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=692x720 at 0x73B69184B4C0>}, {'type': 'text', 'text': 'Detect the bounding box of the nutrition table.'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(10,8),(978,994)<|box_end|>'}]}]

Example 3:
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3120x4208 at 0x73B69184B880>}, {'type': 'text', 'text': 'Detect the bounding box of the nutrition table.'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(320,171),(627,625)<|box_end|>'}]}]

Example 4:
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1944x2592 at 0x73B69184B310>}, {'type': 'text', 'text': 'Detect the bounding box of the nutrition table.'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(143,251),(376,468)<|box_end|>'}]}]

In [ ]:
# Final Code to make sure the data is formatted correctly after dataset.map:


batch = train_dataset_formatted

# Extract messages and images from each sample
messages_list = [sample['messages'] for sample in batch]
images_list = [sample.get('image', None) for sample in batch]

# Filter out samples without images
valid_pairs = [(m, img) for m, img in zip(messages_list, images_list) if img is not None]
if not valid_pairs:
    raise ValueError("Batch contains no valid images.")

messages_list, images_list = zip(*valid_pairs)
messages_list = list(messages_list)
images_list = list(images_list)

# Process each sample
# texts = []
# all_images = []

all_conversations = []  # This will hold complete conversations, each conversation is a list of 3 messages (system, assistant, user)

for messages, image in zip(messages_list, images_list):
    messages_with_image = []
    # Clean up messages: remove None values and restore images
    for msg in messages:
        msg_copy = {'role': msg['role'], 'content': []}
        
        for content_item in msg['content']:
            # Skip None entries entirely
            if content_item is None:
                continue
                
            # Process text content - filter out None text values
            if content_item.get('type') == 'text':
                text_value = content_item.get('text')
                if text_value is not None and text_value != 'None':  # Check for actual None and string 'None'
                    msg_copy['content'].append({
                        'type': 'text',
                        'text': text_value
                    })
            # Process image content - replace IMAGE_PLACEHOLDER with actual PIL image
            elif content_item.get('type') == 'image' and msg['role'] == 'user':
                # Check if it's IMAGE_PLACEHOLDER and replace with actual image
                image_value = content_item.get('image')
                if image_value == 'IMAGE_PLACEHOLDER' or image_value is None:
                    # Replace with the actual PIL image from top level
                    msg_copy['content'].append({
                        'type': 'image',
                        'image': image  # Use the actual PIL image
                    })
                elif image_value and image_value != 'None':
                    # Use existing image if it's not placeholder
                    msg_copy['content'].append({
                        'type': 'image',
                        'image': image_value
                    })
        
        if msg_copy['content']:
            messages_with_image.append(msg_copy)

    # Add the complete conversation (3 messages) to collection
    all_conversations.append(messages_with_image)

print("First 5 all_conversations examples:")
for i in range(min(5, len(all_conversations))):
    print(f"\nExample {i}:")
    print(all_conversations[i])

First 5 all_conversations examples:

Example 0:
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2592x1944 at 0x73AB0C0E8400>}, {'type': 'text', 'text': 'Detect the bounding box of the nutrition table.'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(14,57),(991,603)<|box_end|>'}]}]

Example 1:
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=306x408 at 0x73AB0C0E83D0>}, {'type': 'text', 'text': 'Detect the bounding box of the nutrition table.'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(147,151),(516,588)<|box_end|>'}]}]

Example 2:
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=692x720 at 0x73AB0C0E84F0>}, {'type': 'text', 'text': 'Detect the bounding box of the nutrition table.'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(10,8),(978,994)<|box_end|>'}]}]

Example 3:
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3120x4208 at 0x73AB0C0E8670>}, {'type': 'text', 'text': 'Detect the bounding box of the nutrition table.'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(320,171),(627,625)<|box_end|>'}]}]

Example 4:
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1944x2592 at 0x73AB0C0E86D0>}, {'type': 'text', 'text': 'Detect the bounding box of the nutrition table.'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(143,251),(376,468)<|box_end|>'}]}]

In [ ]:
# put a batch in apply_chat_template 

text = processor.apply_chat_template(
    all_conversations,
    tokenize=False,
    add_generation_prompt=False
)

for i in range(2):
    print(text[i])
    print("-"*100)

print(type(text))

<|im_start|>system
You are a Vision Language Model specialized in interpreting visual data from product images.
Your task is to analyze the provided product images and detect the nutrition tables in a certain format.
Focus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>Detect the bounding box of the nutrition table.<|im_end|>
<|im_start|>assistant
<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(14,57),(991,603)<|box_end|><|im_end|>

----------------------------------------------------------------------------------------------------
<|im_start|>system
You are a Vision Language Model specialized in interpreting visual data from product images.
Your task is to analyze the provided product images and detect the nutrition tables in a certain format.
Focus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>Detect the bounding box of the nutrition table.<|im_end|>
<|im_start|>assistant
<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(147,151),(516,588)<|box_end|><|im_end|>

----------------------------------------------------------------------------------------------------
<class 'list'>

In [ ]:
image, video = process_vision_info(all_conversations)
for i in range(5):
    print(image[i])
    print("-"*100)
# print(video[0:5])

print(type(image))

<PIL.Image.Image image mode=RGB size=2604x1932 at 0x73B60D6310F0>
----------------------------------------------------------------------------------------------------
<PIL.Image.Image image mode=RGB size=308x420 at 0x73B60D6B24D0>
----------------------------------------------------------------------------------------------------
<PIL.Image.Image image mode=RGB size=700x728 at 0x73B60D631000>
----------------------------------------------------------------------------------------------------
<PIL.Image.Image image mode=RGB size=3080x4144 at 0x73B69285DE10>
----------------------------------------------------------------------------------------------------
<PIL.Image.Image image mode=RGB size=1932x2604 at 0x73B6925AFEB0>
----------------------------------------------------------------------------------------------------
<class 'list'>

In [ ]:
batch_inputs = processor(
    text=text,
    images=image,
    # videos=all_videos,
    padding=True,
    truncation=False,
    return_tensors="pt"
)

In [ ]:
print(f'batch_inputs: \n {batch_inputs["input_ids"][0:2, -10:]}') ## First 2 samples, last 10 tokens
print(f'batch_inputs["attention_mask"]: \n {batch_inputs["attention_mask"][0:2, -10:]}') ## 1=real token, 0=padding

print(f'batch_inputs["input_ids"]: \n {batch_inputs["input_ids"][0:2, 0:10]}') # First 2 samples, first 10 tokens
print(f'batch_inputs["attention_mask"]: \n {batch_inputs["attention_mask"][0:2, 0:10]}')

print("-"*100)

print("Input shapes:")
for key, value in batch_inputs.items():
    if hasattr(value, 'shape'):
        print(f"{key}: {value.shape}")

print("-"*100)
print(f'type(batch_inputs): \n {type(batch_inputs)}')

batch_inputs: 
 tensor([[    24,     16,     11,     21,     15,     18,      8, 151649, 151645,
            198],
        [    16,     21,     11,     20,     23,     23,      8, 151649, 151645,
            198]])
batch_inputs["attention_mask"]: 
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
batch_inputs["input_ids"]: 
 tensor([[151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643],
        [151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643]])
batch_inputs["attention_mask"]: 
 tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
----------------------------------------------------------------------------------------------------
Input shapes:
input_ids: torch.Size([1083, 16433])
attention_mask: torch.Size([1083, 16433])
pixel_values: torch.Size([31255872, 1176])
image_grid_thw: torch.Size([1083, 3])
----------------------------------------------------------------------------------------------------
type(batch_inputs): 
 <class 'transformers.feature_extraction_utils.BatchFeature'>

In [ ]:
# __init__ method
print(
    f"processor.tokenizer.pad_token_id: {processor.tokenizer.pad_token_id}\n"
    f"processor.tokenizer.convert_tokens_to_ids('<|vision_start|>'): {processor.tokenizer.convert_tokens_to_ids('<|vision_start|>')}\n"
    f"processor.tokenizer.convert_tokens_to_ids('<|vision_end|>'): {processor.tokenizer.convert_tokens_to_ids('<|vision_end|>')}\n"
    f"processor.tokenizer.convert_tokens_to_ids('<|image_pad|>'): {processor.tokenizer.convert_tokens_to_ids('<|image_pad|>')}"
)

processor.tokenizer.pad_token_id: 151643
processor.tokenizer.convert_tokens_to_ids('<|vision_start|>'): 151652
processor.tokenizer.convert_tokens_to_ids('<|vision_end|>'): 151653
processor.tokenizer.convert_tokens_to_ids('<|image_pad|>'): 151655

Also, the following is the context of model and tokenizer config.

In [ ]:
# CRITICAL: Analyze the tokenizer and its special tokens and model.config
print("\n" + "=" * 80)
print("TOKENIZER ANALYSIS - Understanding Special Tokens and Configuration")
print("=" * 80)

from transformers import Qwen2VLProcessor
import json

# Load the processor and tokenizer
processor = Qwen2VLProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")
tokenizer = processor.tokenizer

print("\n1. VOCABULARY INFORMATION:")
print("-" * 40)
print(f"Vocabulary size: {tokenizer.vocab_size}")
print(f"Model max length: {tokenizer.model_max_length}")
print(f"Padding side: {tokenizer.padding_side}")
print(f"Truncation side: {tokenizer.truncation_side}")

print(f"len(tokenizer): {len(tokenizer)}")

print("\n2. SPECIAL TOKENS MAP:")
print("-" * 40)
special_tokens_map = tokenizer.special_tokens_map
for token_name, token_value in special_tokens_map.items():
    token_id = tokenizer.convert_tokens_to_ids(token_value) if isinstance(token_value, str) else None
    print(f"{token_name}: '{token_value}' -> ID: {token_id}")

print("\n3. ALL SPECIAL TOKENS:")
print("-" * 40)
all_special_tokens = tokenizer.all_special_tokens
print(f"Total special tokens: {len(all_special_tokens)}")
for i, token in enumerate(all_special_tokens[:20]):  # Show first 20
    token_id = tokenizer.convert_tokens_to_ids(token)
    print(f"  {i+1}. '{token}' -> ID: {token_id}")
if len(all_special_tokens) > 20:
    print(f"  ... and {len(all_special_tokens) - 20} more")

print("\n4. IMPORTANT TOKEN IDS:")
print("-" * 40)
print(f"pad_token_id: {tokenizer.pad_token_id} ('{tokenizer.pad_token}' if exists)")
print(f"bos_token_id: {tokenizer.bos_token_id} ('{tokenizer.bos_token}' if exists)")
print(f"eos_token_id: {tokenizer.eos_token_id} ('{tokenizer.eos_token}' if exists)")
print(f"unk_token_id: {tokenizer.unk_token_id} ('{tokenizer.unk_token}' if exists)")

# Check for vision-specific tokens
print("\n5. VISION-SPECIFIC TOKENS:")
print("-" * 40)
vision_tokens = ["<|vision_start|>", "<|vision_end|>", "<|image_pad|>", "<|video_pad|>", 
                 "<|object_ref_start|>", "<|object_ref_end|>", "<|box_start|>", "<|box_end|>",
                 "<|im_start|>", "<|im_end|>"]
for token in vision_tokens:
    if token in tokenizer.get_vocab():
        token_id = tokenizer.convert_tokens_to_ids(token)
        print(f"'{token}' -> ID: {token_id}")
    else:
        print(f"'{token}' -> NOT IN VOCABULARY")

print("\n6. CHAT TEMPLATE:")
print("-" * 40)
if hasattr(tokenizer, 'chat_template'):
    print("Chat template exists:")
    # Print first 500 chars of template
    template_str = str(tokenizer.chat_template)[:500]
    print(template_str + "..." if len(str(tokenizer.chat_template)) > 500 else template_str)
else:
    print("No chat template found")

print("\n7. TOKEN ID RANGE CHECK:")
print("-" * 40)
print(f"Valid token ID range: 0 to {tokenizer.vocab_size - 1}")
print(f"Any token ID >= {tokenizer.vocab_size} will cause the training error!")



# Check Model 

import torch, transformers

# 2) Model: use the correct family for VL

from transformers import Qwen2VLForConditionalGeneration
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    torch_dtype=torch.bfloat16, # original training precision stored in config.json for this model is bfloat16, so "auto" = torch.bfloat16 for this model
    attn_implementation="flash_attention_2",
    device_map="cuda"
)

# 3) Collect sizes (read-only)
tok_vocab_size = tokenizer.vocab_size
tok_len        = len(tokenizer)
added_vocab    = tokenizer.get_added_vocab()
n_added        = len(added_vocab)

emb_in  = model.get_input_embeddings().weight.shape[0]
emb_out = (model.get_output_embeddings().weight.shape[0]
           if model.get_output_embeddings() is not None else None)
cfg_vocab = getattr(model.config, "vocab_size", None)

print("---- TOKENIZER ----")
print(f"tokenizer.vocab_size : {tok_vocab_size}")
print(f"len(tokenizer)       : {tok_len}  (added tokens: {n_added})")

print("\n---- MODEL ----")
print(f"config.vocab_size    : {cfg_vocab}")
print(f"input emb rows       : {emb_in}")
print(f"lm_head rows         : {emb_out}")
print(f"config class         : {model.config.__class__.__name__}")

================================================================================
TOKENIZER ANALYSIS - Understanding Special Tokens and Configuration
================================================================================
/home/zhuoyuan/miniconda3/envs/vlm_Qwen2VL_object_detection/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.

1. VOCABULARY INFORMATION:
----------------------------------------
Vocabulary size: 151643
Model max length: 32768
Padding side: left
Truncation side: right
len(tokenizer): 151657

2. SPECIAL TOKENS MAP:
----------------------------------------
eos_token: '<|im_end|>' -> ID: 151645
pad_token: '<|endoftext|>' -> ID: 151643
additional_special_tokens: '['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']' -> ID: None

3. ALL SPECIAL TOKENS:
----------------------------------------
Total special tokens: 14
  1. '<|im_end|>' -> ID: 151645
  2. '<|endoftext|>' -> ID: 151643
  3. '<|im_start|>' -> ID: 151644
  4. '<|object_ref_start|>' -> ID: 151646
  5. '<|object_ref_end|>' -> ID: 151647
  6. '<|box_start|>' -> ID: 151648
  7. '<|box_end|>' -> ID: 151649
  8. '<|quad_start|>' -> ID: 151650
  9. '<|quad_end|>' -> ID: 151651
  10. '<|vision_start|>' -> ID: 151652
  11. '<|vision_end|>' -> ID: 151653
  12. '<|vision_pad|>' -> ID: 151654
  13. '<|image_pad|>' -> ID: 151655
  14. '<|video_pad|>' -> ID: 151656

4. IMPORTANT TOKEN IDS:
----------------------------------------
pad_token_id: 151643 ('<|endoftext|>' if exists)
bos_token_id: None ('None' if exists)
eos_token_id: 151645 ('<|im_end|>' if exists)
unk_token_id: None ('None' if exists)

5. VISION-SPECIFIC TOKENS:
----------------------------------------
'<|vision_start|>' -> ID: 151652
'<|vision_end|>' -> ID: 151653
'<|image_pad|>' -> ID: 151655
'<|video_pad|>' -> ID: 151656
'<|object_ref_start|>' -> ID: 151646
'<|object_ref_end|>' -> ID: 151647
'<|box_start|>' -> ID: 151648
'<|box_end|>' -> ID: 151649
'<|im_start|>' -> ID: 151644
'<|im_end|>' -> ID: 151645

6. CHAT TEMPLATE:
----------------------------------------
Chat template exists:
{% set image_count = namespace(value=0) %}{% set video_count = namespace(value=0) %}{% for message in messages %}{% if loop.first and message['role'] != 'system' %}<|im_start|>system
You are a helpful assistant.<|im_end|>
{% endif %}<|im_start|>{{ message['role'] }}
{% if message['content'] is string %}{{ message['content'] }}<|im_end|>
{% else %}{% for content in message['content'] %}{% if content['type'] == 'image' or 'image' in content or 'image_url' in content %}{% set image_count.value = im...

7. TOKEN ID RANGE CHECK:
----------------------------------------
Valid token ID range: 0 to 151642
Any token ID >= 151643 will cause the training error!

---- TOKENIZER ----
tokenizer.vocab_size : 151643
len(tokenizer)       : 151657  (added tokens: 14)

---- MODEL ----
config.vocab_size    : 152064
input emb rows       : 152064
lm_head rows         : 152064
config class         : Qwen2VLConfig